```cpp
qDebug("Writing multithreaded applications in Qt");
```

<br />
<sub>(CC0) 2017 Manuel Reithuber</sub>


# QThread

Don't do this[<sup>1</sup>][1]

```cpp
class MyWorker: public QThread {
public:
  void run() {
    qDebug("I'm in a thread!");
  }
};

void somewhereInTheMainThread() {
  MyWorker *worker = new MyWorker();
  worker -> start();
}
```

[1]: http://blog.qt.io/blog/2010/06/17/youre-doing-it-wrong/

<sup>1</sup>: http://blog.qt.io/blog/2010/06/17/youre-doing-it-wrong/

Instead do this:

```cpp
class MyWorker: public QObject {
public slots:
  void run() {
    qDebug("I'm also in a thread");
  }
};

void somewhereInTheMainThread() {
  QThread *thread = new QThread();
  MyWorker worker = new MyWorker();

  worker->moveToThread(thread);
  connect(thread, SIGNAL(started()),
    worker, SLOT(run());

  thread->start();
}
```

Or use `QThreadPool`:

```cpp
class MyWorker: public QRunnable {
public:
  void run() {
    for (int i = 0; i < 100; i += 10) {
      emit progress(i);
      sleep(1);
    }
    emit done()
  }
signals:
  void progress(int); // in percent
  void done()
};

void somewhereInTheMainThread() {
  QThreadPool *pool = new QThreadPool()
  pool->setMaxThreadCount(10);
  pool->start(new MyWorker());
}
```

# Writing thread safe code

  ## Signals/Slots

```cpp
connect(
  srcObject, SIGNAL(somethingHappened()),
  tgtObject, SLOT(react(),
  connectionType);
```

e.g.:

```cpp
QTimer *timer = new QTimer(this);
timer.setInterval(500);
connect(timer, SIGNAL(timeout(), this, SLOT(onTimeout());
timer.start();
```

basically a (per-thread) message bus

`Qt::DirectConnection`
- 'simply' calls the slot method

`Qt::QueuedConnection`
- puts the event into the target thread's event queue


`Qt::AutoConnection`
- Picks the right one from above based on whether the objects belong to the same thread

```cpp
Q_DECLARE_METATYPE(...);
qRegisterMetatype(...);
```

# Mutexes and locks

`QMutex`, `QSemaphore`, `QReadWriteLock`, ...

```cpp
class Threadsafe {
  QMutex m_mutex;
  int m_value;
public:
  void increment() {
    QMutexLocker l(&m_mutex);
    m_value = getValue() + 1;
  }

  int getValue() {
    QMutexLocker l(&m_mutex);
    return m_value;
  }
};
```

... will lock forever (unless you set `m_mutex`'s RecursionMode to `QMutex::Recursive`)

```cpp
class Threadsafe {
  QReadWriteLock m_lock(QReadWriteLock::Recursive);
  int m_value;
public:
  void increment() {
    QWriteLocker l(&m_lock);
    m_value = getValue() + 1;
  }

  QDateTime getValue() {
    QReadLocker l(&m_lock)
    return m_value;
  }
};
```

... will lock forever (relocking is not supported - not even if we already have the write lock)

## Deadlocks

```cpp
QMutex m1, m2;

void a() {
  m1.lock();
  // fetch res1
  m2.lock();
  // operate on res1+res2
  m2.unlock(); m1.unlock();
}

void b() {
  m2.lock();
  // fetch res2
  m1.lock();
  // operate on res1+res2
  m1.unlock(); m2.unlock();
}
```

## Immutability

```cpp
class MyData {
public:
  MyData(const QString &name): m_name(name) {}

private:
  const QString m_name;
}
```

(marking fields `const` is not a must, but makes things cleaner)

## `QAtomicInt`

Provide lock-free atomic operations (great for reference counting)

Operations:
- `fetchAndAdd()`
- `testAndSet()`
- `ref()` and `deref()` (return `true` if new value is nonzero)

# Introspection

- `QThread::currentThread()`
- `QThread.setObjectName()`
- `QObject.thread()`

```cpp
void MyClass::someMethod() {
  if (this->thread() != QThread::currentThread()) {
    qFatal("PANIC!!!!");
  }

  // ... actual code ...
}
```

## `qt-prof`

[github.com/mreithub/qt-prof](https://github.com/mreithub/qt-prof)

Macros:

- `PROFILER(scope, name)`  
  tracks invocation count and execution time of your methods (by QThread)
- `CHECK_THREAD`
  makes sure a method is called from the right QThread (must be in subclass of QObject) 

Won't be compiled unless you define the `USE_PROFILER` macro 

### ... demo time ...

# Subtle pitfalls

- QObject belongs to the wrong thread (e.g. when allocated in the constructor and forgot to call `moveToThread()`
- doing GUI stuff outside the main thread
- `QThread::run()` vs. `QThread::start()`
- too many threads
- parent in different thread
- `delete` vs. `deleteLater()`
- `QNetworkAccessManager`'s request limit
- race conditions

# Tips
- Keep background tasks as clean and short as possible
- Keep your data objects dumb and immutable (avoid locking unless absolutely necessary)
- Use Qt's Signal/Slot system (and pass those immutable objects)
- Keep locks as short as possible
- QMutex::RecursionMode (take special care of `QReadWriteLock` though)
- Use an in-app DB (like SQLite)


# Questions?

<br />
<br />

- Twitter: @mreithub
- Github: mreithub
- Blog: //manuel.reithuber.net/
